# 6. Beam Search and BLEU score

## 강의 소개

문장을 decoding 하는 데에 사용하는 대표적인 알고리즘인 **Beam Search**와 번역 task에서 번역된 문장을 평가하는 대표적인 metric인 **BLEU score**를 소개합니다.

**Beam Search**

- 언어 모델이 문장을 generation할 때에는 확률값에 기반한 다양한 경우의 수가 존재합니다.
- 모든 경우의 수를 고려하는 것은 비효율적이며 너무 작은 확률값까지 고려한다면 생성된 문장의 quality가 떨어질 수 있습니다.
- 가장 높은 확률값을 고려하는 방법 역시 모델이 단순한 generation을 하도록 하게 만드는 단점이 있을 수 있습니다.
- 이러한 문제의 대안으로 제안된 Beam Search를 알아봅니다.

**BLEU score**

- 자연어는 컴퓨터가 이해할 수 있는 방식으로 변환되어 모델의 입력 및 출력으로 활용되기 때문에 적절한 metric을 이용해 모델을 평가해야 합니다.
- 다양한 자연어처리 관련 metric이 있지만, 그중에서도 번역 task에서 가장 대표적인 BLEU score를 소개합니다.
- 번역에 있어서 BLEU score가 precision을 고려하는 이유에 대해서 고민하면서 강의를 들어주시면 좋을 것 같습니다.

<br>

## Further Reading

- [Deep learning.ai-BeamSearch](https://www.youtube.com/watch?v=RLWuzLLSIgw&feature=youtu.be)
- [Deep learning.ai-RefiningBeamSearch](https://www.youtube.com/watch?v=gb__z7LlN_4&feature=youtu.be)
- [OpenNMT-beam search](https://opennmt.net/OpenNMT/translation/beam_search/)

<br>

## Further Question

- BLEU score가 번역 문장 평가에 있어서 갖는 단점은 무엇이 있을까요?
  - 참고 : [Tangled up in BLEU: Reevaluating the Evaluation of Automatic Machine Translation Evaluation Metrics](https://arxiv.org/abs/2006.06264?utm_source=feedburner&utm_medium=feed&utm_campaign=Feed%253A+arxiv%252FQSXk+%2528ExcitingAds%2521+cs+updates+on+arXiv.org%2529)

<br>

## 6.1 Beam Search

### References

- [Machine Translation, Sequence-to-sequence and Attention](https://web.stanford.edu/class/cs224n/slides/cs224n-2019-lecture08-nmt.pdf)
- [[Sooftware 머신러닝] Beam Search (빔서치)](https://blog.naver.com/PostView.nhn?blogId=sooftware&logNo=221809101199&from=search&redirect=Log&widgetTypeCall=true&directAccess=false)

<br>

### 6.1.1 Greedy decoding

- 디코더에서 하나의 단어만을 입력으로 사용하여 다음 단어를 예측하는 것을 Greedy decoding 이라고 한다.
- Greedy decoding has no way to undo decisions!

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=11W2qMRev0hbM_e-Qeps9AR-Wcf5jUAAz' width=500/>

- How can we fix this?

<br>

### 6.1.2 Exhaustive search (철저한 탐색)

- Ideally, we want to find a (length $T$) translation $y$ that maximizes
- $x$: 입력 문장
- $y$: 출력 문장

<br>

\begin{align*}
P(y \mid x)&=P\left(y_{1} \mid x\right) P\left(y_{2} \mid y_{1}, x\right) P\left(y_{3} \mid y_{2}, y_{1}, x\right) \ldots P\left(y_{T} \mid y_{1}, \ldots, y_{T-1}, x\right)\\
&=\prod_{1}^{T} P\left(y_{t} \mid y_{1}, \ldots, y_{t-1}, x\right)
\end{align*}

<br>

- We could try computing <font color='red'>all possible sequences</font> $y$
  - This means that on each step $t$ of the decoder, we are tracking $V^t$ possible partial translations, where $V$ is the vocabulary size
  - This $O(V^t)$ complexity is far too expensive!
- Exhaustive search 의 차선책으로 등장한 것이 Beam Search 이다.

<br>

### 6.1.3 Beam search

- Greedy decoding 과 Exhaustive search 의 중간쯤에 있는 아이디어

<br>

#### 6.1.3.1 Core idea

- on each time step of the decoder, we keep track of the <font color='red'>$k$</font> most probable partial translations (which we call hypothese)
- 매 time step 마다 미리 정해놓은 $k$개의 단어를 고려하여 예측을 수행한다.
- $k$개의 candidate 중에서 가장 확률이 높은 것을 택한다.
  - $k$ is the beam size (in practice around 5 to 10)

<br>

- A hypothesis $y_1, \dots, y_t$ has a score of its log probability:

$$
\operatorname{score}\left(y_{1}, \ldots, y_{t}\right)=\log P_{L M}\left(y_{1}, \ldots, y_{t} \mid x\right)=\sum_{i=1}^{t} \log P_{L M}\left(y_{i} \mid y_{1}, \ldots, y_{i-1}, x\right)
$$

- 최대화하고자 하는 값은 joint probability $P_{L M}\left(y_{1}, \ldots, y_{t} \mid x\right)$ 이다.
- 이 값에 로그를 씌우게 되면 확률값들의 덧셈으로 변환할 수 있다.
- 여기서 사용된 로그함수는 단조증가함수이기 때문에 적용하기 전에 가장 큰 확률값도 로그 적용 후 큰 값으로 유지될 수 있다.

<br>

- Scores are all negative, and a higher score is better
- We search for high-scoring hypotheses, tracking the top $k$ ones on each step

<br>

#### 6.1.3.2 Caution

- Beam search is <font color='red'>not guaranteed</font> to find a globally optimal solution. 
- But it is <font color='red'>much more efficient</font> than exhaustive search!


<br>

#### 6.1.3.3 Example

- Beam size: $k=2$

<br>

- Calculate prob dist of next word

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=11mba4LZjqoThQP6VjlvLhePdhoxqGqCt' width=800/>

<br>

- Take top k words and compute scores
- 예측값은 vocabulary 상에 정의된 단어의 확률분포로 output이 나타나게 된다.
- 가장 확률값이 높은 2개의 단어를 선택하게 된다.
- 확률값들은 0~1 사이의 값들을 갖게 되는데 로그함수의 x절편이 1이기 때문에 로그를 취한 확률값은 모두 음수로 나오게 된다.
- 확률값이 커질수록 로그를 씌운 확률값은 0에 가깝게 나타난다.
  - 즉, 결과값이 음수 중에 0에 가까운 값일수록 큰 확률을 갖는다고 볼 수 있다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=11otZ_NmjTHpZmwa6aPZuqlcxnCYQaLSV' width=800/>

<br>

- For each of the $k$ hypotheses, find top $k$ next words and calculate scores
- 일시적으로 $k^k$개(=4)의 hypothesis를 고려해서 각각의 자식 노드의 예측값을 생성한다.
- 해당 예측값과 부모노드의 예측값의 합이 가장 큰값을 갖는 beam size(=2) 만큼의 hypothesis를 선택한다.


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=11wR43z_KvBkGjNWWmImfrUsRfCfd7g6W' width=800/>

<br>

- For each of the $k$ hypotheses, find top $k$ next words and calculate scores

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=121PeRYzdnvRvmdpddZbP4piMjRFZRjLc' width=800/>

<br>

- For each of the $k$ hypotheses, find top $k$ next words and calculate scores

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=126bgbAdY9p4c_b_b99wovkJHgQkO60S4' width=800/>

<br>

- For each of the $k$ hypotheses, find top $k$ next words and calculate scores

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=12EP4BSxVhE3BEcOzFvvpWT5HiMVUOGNe' width=800/>

<br>

- For each of the $k$ hypotheses, find top $k$ next words and calculate scores

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=12RkSWhURqPR7zoNLg7oVr-ezj7fn2XGz' width=800/>

<br>

#### 6.1.3.4 Stopping criterion

- In <font color='skyblue'>greedy decoding</font>, usually we decode until the model produces a <font color='red'>\<END\> token</font>
  - For example: `<START> he hit me with a pie <END>`

<br>

- In <font color='skyblue'>beam search decoding</font>, different hypotheses may produce \<END\> tokens on <font color='red'>different timesteps</font>
- 각각의 hypothesis들은 서로 다른 time step에서 `<END>` 토큰을 만날 수가 있다.
  - When a hypothesis produces \<END\>, that hypothesis is <font color='red'>complete</font>
  - `<END>` 토큰을 만난 hypothesis에 대해서는 생성과정을 중단한다.
  - <font color='red'>Place ist aside</font> and continue exploring other hypotheses via beam search
  - 완료된 hypothesis는 임시 공간에 저장한 뒤 나머지 hypothesis들의 생성과정을 진행한다.

<br>

- Usually we continue beam search until:
  - We reach timestep $T$ (where $T$ is some pre-defined cutoff), or
    - 특정 time step에 도달했을 때 종료
  - We have at least $n$ completed hypotheses (where $n$ is the pre-defined cutoff)
    - 저장공간에 특정 갯수 이상의 hypothesis가 채워졌을 때 종료

<br>

#### 6.1.3.5 Finishing up

- We have our list of completed hypotheses
- How to select the top one with the highest score?
- Each hypothesis $y_1, \dots, y_t$ on our list has a score

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=12c0fPOs57TW4YjTMyssgwmFK9YA_ey38' width=600/>

- Problem with this: <font color='red'>longer hypotheses have lower scores</font>
  - hypothesis들의 시퀀스의 길이가 서로 다를 수가 있다.
  - 상대적으로 짧은 길이를 갖는 시퀀스가 높은 joint probability 값을 갖게될 것이다.
  - 긴 길이의 시퀀스를 갖는 hypothesis는 낮은 joint probability 값을 갖게 된다.
- Fix: Normalize by length
  - joint probability 의 값을 시퀀스의 길이로 나눠줌으로서 normalize 할 수 있다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=12br5XIulIhHxxgVg-niNxMKKW2-CZOrc' width=450/>

<br>

## 6.2 BLEU score

### References

- [딥러닝을 이용한 자연어 처리 입문 - BLEU Score](https://wikidocs.net/31695)

<br>

### 6.2.1 Precision and Recall

#### 6.2.1.1 Example 1

- Reference: <font color='green'>Half</font> of <font color='green'>my heart is in</font> Havana <font color='green'>ooh na</font> na
- Predicted: <font color='green'>Half</font> as <font color='green'>my heart is in</font> Obama <font color='green'>ooh na</font>

- precision: 정밀도

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=12dJ46YqlxM0DLPf2AjBU_BlTGPOxAUD0' width=500/>

- 양성으로 예측한 것들 중 양성으로 예측을 성공한 비율
- 예측된 결과가 도출이 되었을 때 실질적으로 느끼는 정확도
  - `length_of_prediction` : predicted 의 단어 수
  - `#(correct words)` : predicted 기준 reference 와 겹치는 단어 수



- recall: 재현율

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=12gNGpdqvUHmfIoRzDcDCo30GZGJ201Hu' width=500/>

- 실제 양성인 것들 중 양성으로 예측을 성공한 비율
- 실제값에 대한 정확도
  - `length_of_reference` : reference 의 단어 수
  - `#(correct words)` : predicted 기준 reference 와 겹치는 단어 수



- F - measure
- precision 과 recall 의 조화 평균
  - 산술평균 > 기하평균 > 조화평균
  - 조화평균은 작은 값에 더 큰 가중치를 부여한 형태로 볼 수 있다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=12h0pjN8OXakGcK8EhTiLijQDLuVMelLS' width=600/>

<br>

#### 6.2.1.2 Example 2

- Predicted (from model 1): <font color='green'>Half</font> as <font color='green'>my heart is in</font> Obama <font color='green'>ooh na</font>
- Reference: $\qquad\qquad\quad\;\;$Half of my heart is in Havana ooh na na
- Predicted (from model 2): <font color='green'>Havana na in heart my is Half ooh of na</font>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=12icRh9VDJUyeJJVkH_N7xd9od2GUEGpD' width=600/>

- <font color='red'>Flaw(결함): no penalty for reordering</font>
  - precision과 recall을 통한 모델 평가는 문장의 순서에 따른 패널티를 부과할 수 없다.


<br>

### 6.2.2 BiLingual Evaluation Understudy (BLEU) score

- <font color='red'>N-gram overlap</font> between machine translation output and reference sentence
  - 개별 단어 레벨에서 봤을 때 얼마나 ground truth 문장과 겹치는 단어들이 나왔는가를 평가할 뿐만 아니라 **n-gram을 통해 연속된 단어 집합에 대한 평가**도 함께 실시한다.

- Compute preecision for <font color='red'>n-grams of size one to four(1~4)</font>
  - BLEU는 precision 만을 고려한다. (recall은 무시)
  - 이는 번역 결과만을 보고 정확도를 판단한다고 볼 수 있다.

- Add brevity(간결성) penalty (for too short translations)
  - reference의 길이 보다 짧은 문장을 생성한 경우에 짧아진 비율만큼 precision의 기하평균값을 낮춰준다는 의미이다.
  - brevity penalty는 recall의 최대값을 의미한다.
  - Typically computed over the entire corpus, not on single sentences

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=130fc13BO53Z9OtSjoqxbpOpE_k3yCSyi' width=600/>

- precision의 기하평균
  - 1-gram 부터 4-gram 까지의 precision을 구한 후 이들의 기하평균을 계산한다.


<br>

- Predicted (from model 1): Half as <font color='green'>my heart is in</font> Obama ooh na
- Reference: $\qquad\qquad\quad\;\;$Half of my heart is in Havana ooh na na
- Predicted (from model 2): Havana na in heart my is Half ooh of na

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=12oSkfoKqopuCk_1zuIn7Cs-cjHki4gG9' width=600/>

<br>

## 6.3 실습: Seq2seq + Attention

1. 여러 Attention 모듈을 구현한다.
2. 기존 Seq2seq 모델과의 차이를 이해한다.

<br>

### 6.3.1 필요 패키지 import

In [1]:
from tqdm import tqdm
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.nn import functional as F

import torch
import random

<br>

### 6.3.2 데이터 전처리

- 데이터 전처리는 이전과 동일하다.

In [2]:
vocab_size = 100
pad_id = 0
sos_id = 1
eos_id = 2

src_data = [
  [3, 77, 56, 26, 3, 55, 12, 36, 31],
  [58, 20, 65, 46, 26, 10, 76, 44],
  [58, 17, 8],
  [59],
  [29, 3, 52, 74, 73, 51, 39, 75, 19],
  [41, 55, 77, 21, 52, 92, 97, 69, 54, 14, 93],
  [39, 47, 96, 68, 55, 16, 90, 45, 89, 84, 19, 22, 32, 99, 5],
  [75, 34, 17, 3, 86, 88],
  [63, 39, 5, 35, 67, 56, 68, 89, 55, 66],
  [12, 40, 69, 39, 49]
]

trg_data = [
  [75, 13, 22, 77, 89, 21, 13, 86, 95],
  [79, 14, 91, 41, 32, 79, 88, 34, 8, 68, 32, 77, 58, 7, 9, 87],
  [85, 8, 50, 30],
  [47, 30],
  [8, 85, 87, 77, 47, 21, 23, 98, 83, 4, 47, 97, 40, 43, 70, 8, 65, 71, 69, 88],
  [32, 37, 31, 77, 38, 93, 45, 74, 47, 54, 31, 18],
  [37, 14, 49, 24, 93, 37, 54, 51, 39, 84],
  [16, 98, 68, 57, 55, 46, 66, 85, 18],
  [20, 70, 14, 6, 58, 90, 30, 17, 91, 18, 90],
  [37, 93, 98, 13, 45, 28, 89, 72, 70]
]

In [3]:
trg_data = [[sos_id] + seq + [eos_id] for seq in tqdm(trg_data)]

100%|██████████| 10/10 [00:00<00:00, 13374.69it/s]


In [4]:
def padding(data, is_src=True):
    max_len = len(max(data, key=len))
    print(f"Maximum sequence length: {max_len}")

    valid_lens = []
    for i, seq in enumerate(tqdm(data)):
        valid_lens.append(len(seq))
        if len(seq) < max_len:
            data[i] = seq + [pad_id] * (max_len - len(seq))

    return data, valid_lens, max_len

In [5]:
src_data, src_lens, src_max_len = padding(src_data)
trg_data, trg_lens, trg_max_len = padding(trg_data)

Maximum sequence length: 15


100%|██████████| 10/10 [00:00<00:00, 65027.97it/s]


Maximum sequence length: 22


100%|██████████| 10/10 [00:00<00:00, 21720.89it/s]


In [6]:
# B: batch size, S_L: source maximum sequence length, T_L: target maximum sequence length
src_batch = torch.LongTensor(src_data) # (B, S_L)
src_batch_lens = torch.LongTensor(src_lens) # (B,)
trg_batch = torch.LongTensor(trg_data)  # (B, T_L)
trg_batch_lens = torch.LongTensor(trg_lens)  # (B)

print(src_batch.shape)
print(src_batch_lens.shape)
print(trg_batch.shape)
print(trg_batch_lens.shape)

torch.Size([10, 15])
torch.Size([10])
torch.Size([10, 22])
torch.Size([10])


In [8]:
src_batch_lens, sorted_idx = src_batch_lens.sort(descending=True)
src_batch = src_batch[sorted_idx]
trg_batch = trg_batch[sorted_idx]
trg_batch_lens = trg_batch_lens[sorted_idx]

print(src_batch)
print(src_batch_lens)
print(trg_batch)
print(trg_batch_lens)

tensor([[39, 47, 96, 68, 55, 16, 90, 45, 89, 84, 19, 22, 32, 99,  5],
        [41, 55, 77, 21, 52, 92, 97, 69, 54, 14, 93,  0,  0,  0,  0],
        [63, 39,  5, 35, 67, 56, 68, 89, 55, 66,  0,  0,  0,  0,  0],
        [ 3, 77, 56, 26,  3, 55, 12, 36, 31,  0,  0,  0,  0,  0,  0],
        [29,  3, 52, 74, 73, 51, 39, 75, 19,  0,  0,  0,  0,  0,  0],
        [58, 20, 65, 46, 26, 10, 76, 44,  0,  0,  0,  0,  0,  0,  0],
        [75, 34, 17,  3, 86, 88,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [12, 40, 69, 39, 49,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [58, 17,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [59,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])
tensor([15, 11, 10,  9,  9,  8,  6,  5,  3,  1])
tensor([[ 1, 37, 14, 49, 24, 93, 37, 54, 51, 39, 84,  2,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0],
        [ 1, 32, 37, 31, 77, 38, 93, 45, 74, 47, 54, 31, 18,  2,  0,  0,  0,  0,
          0,  0,  0,  0],
        [ 1, 20, 70, 14,  6, 58, 90,

<br>

### 6.3.3 Encoder 구현

- Encoder 역시 기존 Seq2seq 모델과 동일하다.

In [9]:
embedding_size = 256
hidden_size = 512
num_layers = 2
num_dirs = 2
dropout = 0.1

In [10]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.gru = nn.GRU(
            input_size=embedding_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            bidirectional=True if num_dirs > 1 else False,
            dropout=dropout
        )
        self.linear = nn.Linear(num_dirs * hidden_size, hidden_size)

    def forward(self, batch, batch_lens): # batch: (B, S_L), batch_lens: (B)
        # d_w: word embedding size
        batch_emb = self.embedding(batch)  # (B, S_L, d_w)
        batch_emb = batch_emb.transpose(0, 1)  # (S_L, B, d_w)

        packed_input = pack_padded_sequence(batch_emb, batch_lens)

        h_0 = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size))  # (num_layers*num_dirs, B, d_h) = (4, B, d_h)
        packed_outputs, h_n = self.gru(packed_input, h_0)  # h_n: (4, B, d_h)
        outputs = pad_packed_sequence(packed_outputs)[0]  # outputs: (S_L, B, 2d_h)
        outputs = torch.tanh(self.linear(outputs))  # (S_L, B, d_h)

        forward_hidden = h_n[-2, :, :]
        backward_hidden = h_n[-1, :, :]
        hidden = torch.tanh(self.linear(torch.cat((forward_hidden, backward_hidden), dim=-1))).unsqueeze(0)  # (1, B, d_h)

        return outputs, hidden

In [11]:
encoder = Encoder()

<br>

### 6.3.4 Dot-product Attention 구현

- 우선 대표적인 attention 형태 중 하나인 Dot-product Attention은 다음과 같이 구현할 수 있습니다.

In [12]:
class DotAttention(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, decoder_hidden, encoder_outputs): # (1, B, d_h), (S_L, B, d_h)
        query = decoder_hidden.squeeze(0) # (B, d_h)
        key = encoder_outputs.transpose(0, 1) # (B, S_L, d_h)

        energy = torch.sum(torch.mul(key, query.unsqueeze(1)), dim=-1) # (B, S_L)

        attn_scores = F.softmax(energy, dim=-1) # (B, S_L)
        attn_values = torch.sum(torch.mul(encoder_outputs.transpose(0, 1), attn_scores.unsqueeze(2)), dim=1) # (B, d_h)

        return attn_values, attn_scores

In [13]:
dot_attn = DotAttention()

<br>

### 6.3.5 Decoder 구현 (Dot-product attention)

- Dot-product attention 모듈을 가지는 Decoder 클래스를 구현한다.

In [14]:
class Decoder(nn.Module):
    def __init__(self, attention):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.attention = attention
        self.rnn = nn.GRU(
            embedding_size,
            hidden_size
        )
        self.output_linear = nn.Linear(2*hidden_size, vocab_size)

    def forward(self, batch, encoder_outputs, hidden):  # batch: (B), encoder_outputs: (L, B, d_h), hidden: (1, B, d_h)
        batch_emb = self.embedding(batch) # (B, d_w)
        batch_emb = batch_emb.unsqueeze(0) # (1, B, d_w)

        outputs, hidden = self.rnn(batch_emb, hidden) # (1, B, d_h), (1, B, d_h)

        attn_values, attn_scores = self.attention(hidden, encoder_outputs) # (B, d_h), (B, S_L)
        concat_outputs = torch.cat((outputs, attn_values.unsqueeze(0)), dim=-1) # (1, B, 2d_h)

        return self.output_linear(concat_outputs).squeeze(0), hidden # (B, V), (1, B, d_h)

In [15]:
decoder = Decoder(dot_attn)

<br>

### 6.3.6 Seq2seq 모델 구축

- 최종적으로 seq2seq 모델을 다음과 같이 구성할 수 있다.

In [16]:
class Seq2seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2seq, self).__init__()

        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src_batch, src_batch_lens, trg_batch, teacher_forcing_prob=0.5):
        # src_batch: (B, S_L)
        # src_batch_lens: (B)
        # trg_batch: (B, T_L)

        encoder_outputs, hidden = self.encoder(src_batch, src_batch_lens) # encoder_outputs: (S_L, B, d_h), hidden: (1, B, d_h)

        input_ids = trg_batch[:, 0] # (B)
        batch_size = src_batch.shape[0]
        outputs = torch.zeros(trg_max_len, batch_size, vocab_size) # (T_L, B, V)

        for t in range(1, trg_max_len):
            decoder_outputs, hidden = self.decoder(input_ids, encoder_outputs, hidden) # decoder_outputs: (B, V), hidden: (1, B, d_h)

            outputs[t] = decoder_outputs
            _, top_ids = torch.max(decoder_outputs, dim=-1) # top_ids: (B)

            input_ids = trg_batch[:, t] if random.random() > teacher_forcing_prob else top_ids

        return outputs

In [17]:
seq2seq = Seq2seq(encoder, decoder)

<br>

### 6.3.7 모델 사용해보기

- 만든 모델로 결과 확인

In [18]:
# V: vocab size
outputs = seq2seq(src_batch, src_batch_lens, trg_batch) # (T_L, B, V)

print(outputs)
print(outputs.shape)

tensor([[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         ...,
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]],

        [[-3.9002e-02, -6.0949e-02, -7.3044e-02,  ...,  1.2073e-01,
           1.9356e-02, -1.9067e-01],
         [-8.3504e-03, -8.9391e-02, -4.3635e-02,  ...,  1.7211e-01,
          -4.9343e-02, -1.8412e-01],
         [-1.5035e-02, -9.5775e-02, -6.9908e-02,  ...,  1.3096e-01,
          -2.9351e-03, -2.0875e-01],
         ...,
         [ 8.5860e-03, -8

In [19]:
sample_sent = [4, 10, 88, 46, 72, 34, 14, 51]
sample_len = len(sample_sent)

sample_batch = torch.LongTensor(sample_sent).unsqueeze(0) # (1, L)
sample_batch_len = torch.LongTensor([sample_len]) # (1)

encoder_output, hidden = seq2seq.encoder(sample_batch, sample_batch_len) # hidden: (4, 1, d_h) <- (?) (1, 1, d_h)

In [20]:
hidden.shape

torch.Size([1, 1, 512])

In [21]:
input_id = torch.LongTensor([sos_id]) # (1)
output = []

for t in range(1, trg_max_len):
    decoder_output, hidden = seq2seq.decoder(input_id, encoder_output, hidden)  # decoder_output: (1, V), hidden: (4, 1, d_h) <- (?) (1, 1, d_h)

    _, top_id = torch.max(decoder_output, dim=-1)  # top_ids: (1)

    if top_id == eos_id:
        break
    else:
        output += top_id.tolist()
        input_id = top_id    

In [23]:
print(output)

[80, 7, 38, 57, 27, 84, 84, 38, 63, 55, 38, 63, 48, 57, 42, 82, 18, 18, 60, 70, 18]


<br>

### 6.3.8 Concat Attention 구현 (Bahdanau Attention)

- Bahdanau Attention이라고도 불리는 Concat Attention을 구현해보도록 하겠습니다.  
  - `self.w`: Concat한 query와 key 벡터를 1차적으로 linear transformation.
  - `self.v`: Attention logit 값을 계산.

In [24]:
class ConcatAttention(nn.Module):
  def __init__(self):
    super().__init__()

    self.w = nn.Linear(2*hidden_size, hidden_size, bias=False)
    self.v = nn.Linear(hidden_size, 1, bias=False)

  def forward(self, decoder_hidden, encoder_outputs):  # (1, B, d_h), (S_L, B, d_h)
    src_max_len = encoder_outputs.shape[0]

    decoder_hidden = decoder_hidden.transpose(0, 1).repeat(1, src_max_len, 1)  # (B, S_L, d_h)
    encoder_outputs = encoder_outputs.transpose(0, 1)  # (B, S_L, d_h)

    concat_hiddens = torch.cat((decoder_hidden, encoder_outputs), dim=2)  # (B, S_L, 2d_h)
    energy = torch.tanh(self.w(concat_hiddens))  # (B, S_L, d_h)

    attn_scores = F.softmax(self.v(energy), dim=1)  # (B, S_L, 1)
    attn_values = torch.sum(torch.mul(encoder_outputs, attn_scores), dim=1)  # (B, d_h)

    return attn_values, attn_scores

In [25]:
concat_attn = ConcatAttention()

<br>

### 6.3.9 Decoder 구현 (Concat attention)

In [26]:
class Decoder(nn.Module):
  def __init__(self, attention):
    super().__init__()

    self.embedding = nn.Embedding(vocab_size, embedding_size)
    self.attention = attention
    self.rnn = nn.GRU(
        embedding_size + hidden_size, # (?)
        hidden_size
    )
    self.output_linear = nn.Linear(hidden_size, vocab_size)

  def forward(self, batch, encoder_outputs, hidden):  # batch: (B), encoder_outputs: (S_L, B, d_h), hidden: (1, B, d_h)  
    batch_emb = self.embedding(batch)  # (B, d_w)
    batch_emb = batch_emb.unsqueeze(0)  # (1, B, d_w)

    attn_values, attn_scores = self.attention(hidden, encoder_outputs)  # (B, d_h), (B, S_L)

    concat_emb = torch.cat((batch_emb, attn_values.unsqueeze(0)), dim=-1)  # (1, B, d_w+d_h)

    outputs, hidden = self.rnn(concat_emb, hidden)  # (1, B, d_h), (1, B, d_h)

    return self.output_linear(outputs).squeeze(0), hidden  # (B, V), (1, B, d_h)

In [27]:
decoder = Decoder(concat_attn)

<br>

### 6.3.10 Seq2seq 모델 사용 (concat attention)

In [28]:
seq2seq = Seq2seq(encoder, decoder)

In [29]:
outputs = seq2seq(src_batch, src_batch_lens, trg_batch)

print(outputs)
print(outputs.shape)

tensor([[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.0633, -0.0522,  0.0354,  ..., -0.0638,  0.0769,  0.1471],
         [ 0.0322, -0.0379,  0.0097,  ..., -0.0071,  0.0730,  0.2010],
         [ 0.0110, -0.0533, -0.0111,  ..., -0.0246,  0.0761,  0.1379],
         ...,
         [ 0.0380, -0.0317,  0.0043,  ..., -0.0419,  0.0822,  0.1966],
         [ 0.0526, -0.0004, -0.0139,  ..., -0.0436,  0.0530,  0.1265],
         [ 0.0495, -0.0256,  0.0128,  ..., -0.0171,  0.0848,  0.1625]],

        [[ 0.1842, -0.0340, -0.0502,  ...,  0.0617,  0.1970,  0.0072],
         [-0.0629, -0.0461, -0.2107,  ..., -0